In [2]:
import netket as nk
import json
from qutip import *
import numpy as np
import time
import multiprocessing as mp
from collections import OrderedDict
from pickle import dump
import os
import matplotlib.pyplot as plt
import scipy
from matplotlib import gridspec
from functools import reduce
plt.style.use('seaborn')
from scipy.stats import norm
import sys
sys.path.append("/home/victor1/penalty_method")
import expect_grad_ex 
import vmc_ex
import jax
import pickle
import copy
import optax
import penalty_cost as ex

In [3]:
def CSHam(N, B, Ak):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # Create hamiltonian
    hamiltonian = nk.operator.LocalOperator(hilbertSpace, operators=operators, acting_on=sites, dtype=complex)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

def exactDiagonalization_full(hamiltonian):
    # Changes Hamiltonian to matrix form
    haMatrix = hamiltonian.to_dense()
    # Gets eigenvalues and vectors
    eigenValues, v = np.linalg.eigh(haMatrix)
    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

# Error Calculation (Input: the found state, the state from exact diagonalization, the found energy, the energy from exact diagonalization)
def err_sta(state, edState,N):
    overlap = np.dot(state.conj().reshape(2**N, 1).T, edState.reshape(2**N, 1))
    waveFunctionErr = 1 - (np.linalg.norm(overlap))**2
    return waveFunctionErr

# NetKet RBM with stochastic reconfiguration descent
class RBM:
    def __init__(self, N, hamiltonian, hilbertSpace, machine):
        # Assign inputsv[:, i]
        self.hamiltonian, self.hilbertSpace, self.machine, self.N = hamiltonian, hilbertSpace, machine, N
        # Define sampler
        self.sampler = nk.sampler.MetropolisLocal(hilbert=hilbertSpace)
        # Define optimizer
        self.optimizer = nk.optimizer.Sgd(learning_rate=0.005)
        # Define Stochastic reconfiguration
        self.sr = nk.optimizer.SR(diag_shift=0.01) #diagnol shift, its role as regularizer? seems to take a different form as 
        #compared to the version I have seen
        # Variational state
        self.vs = nk.vqs.MCState(self.sampler, self.machine, n_samples=1000, n_discard_per_chain=100) #discarded number of samples 
        #at the beginning of the MC chain

    # Output is the name of the output file in which the descent data is stored
    def __call__(self, output, state_list, shift_list):
        self.vs.init_parameters(jax.nn.initializers.normal(stddev=0.25))
        gs = vmc_ex.VMC_ex(hamiltonian=self.hamiltonian, optimizer=self.optimizer, variational_state=self.vs, preconditioner=self.sr, 
                   state_list = state_list, shift_list = shift_list)
        # Start timing
        start = time.time()
        # Set the output files as well as number of iterations in the descent
        gs.run(out=output, n_iter=300,show_progress = False)
        end = time.time()
        runTime = end - start
        # Import the data from log file
        data = json.load(open(output + '.log'))
        # Extract the relevant information
        # iters = data["Energy"]["iters"]
        energy_RBM = data["Energy"]["Mean"]["real"] #get the real part of the mean energy
       
        # finalEng = energy_RBM[-1]
        finalEng = reduce(lambda x, y: x if y is None else y, energy_RBM)
        # Get machine statethe state of the machine as an array
        state = self.vs.to_array()
        # Outputs the final energy, the final state, and the runtime
        parameters = self.vs.parameters
        # Outputs the final energy, the final state, and the runtime
        return finalEng, state, parameters

In [4]:
N = 6
Ak = []

alpha = 1   #density of RBM
M = alpha*N
# Constant A
B = 0.95
# Variable A
A = N/2
N0 = N/2
for i in range(N-1):
    # Constant A
    #Ak_i = 1
    # Variable A
    Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)
    
# Define hamiltonian and hilbert space
ha, hi = CSHam(N,B,Ak)

#Exact Diagonalization
#e, v = exactDiagonalization_full(ha)

In [5]:
# we first need to load and construct the previously determined state and construct shift_list and state_list


ma = nk.models.RBM(alpha=1, dtype=complex,use_visible_bias=True, use_hidden_bias=True)
sampler0 = nk.sampler.MetropolisLocal(hilbert=hi)
vs0 = nk.vqs.MCState(sampler0, ma, n_samples=10000, n_discard_per_chain=1000)
#vs0.init_parameters(jax.nn.initializers.normal(stddev=0.25))

fileName = "2021_summer_data/2022_winter_data/post_selection_data/parameter_N6i0j38_best.json"
file = open(fileName,'rb')
a = pickle.load(file)
vs0.parameters = a

sampler1 = nk.sampler.MetropolisLocal(hilbert=hi)
vs1 = nk.vqs.MCState(sampler1, ma, n_samples=10000, n_discard_per_chain=1000)
#vs1.init_parameters(jax.nn.initializers.normal(stddev=0.25))

fileName = "2021_summer_data/2022_winter_data/post_selection_data/parameter_N6i1j41_best.json"
file = open(fileName,'rb')
a = pickle.load(file)
vs1.parameters = a


sampler2 = nk.sampler.MetropolisLocal(hilbert=hi)
vs2 = nk.vqs.MCState(sampler2, ma, n_samples=10000, n_discard_per_chain=1000)
#vs1.init_parameters(jax.nn.initializers.normal(stddev=0.25))

fileName = "2021_summer_data/2022_winter_data/post_selection_data/parameter_N6i2j22_best.json"
file = open(fileName,'rb')
a = pickle.load(file)
vs1.parameters = a



# this is for ground state only
ma = nk.models.RBM(alpha=1, dtype=complex,use_visible_bias=True, use_hidden_bias=True)
sampler = nk.sampler.MetropolisLocal(hilbert=hi)
vs = nk.vqs.MCState(sampler, ma, n_samples=10000, n_discard_per_chain=1000)
#vs.init_parameters(jax.nn.initializers.normal(stddev=0.25))

engs = []
# we first need to loop over the 50 runs, and compare their sampled energies with more (10000 samples) precision
for i in range(50):
    fileName = "2021_summer_data/2022_winter_data/parameter_N6i0j" + str(i) + ".json"
    file = open(fileName,'rb')
    a = pickle.load(file)
    vs.parameters = a
    eng = vs.expect(ha)
    engs.append(eng.mean)
    print(i)

In [9]:
# this is for excited state state only
ma = nk.models.RBM(alpha=1, dtype=complex,use_visible_bias=True, use_hidden_bias=True)
sampler = nk.sampler.MetropolisLocal(hilbert=hi)
vs = nk.vqs.MCState(sampler, ma, n_samples=10000, n_discard_per_chain=1000)
#vs.init_parameters(jax.nn.initializers.normal(stddev=0.25))
state_list = [vs0,vs1]
shift_list = [0.3, 0.3]
engs = []
# we first need to loop over the 50 runs, and compare their sampled energies with more (10000 samples) precision
for i in range(50):
    fileName = "2021_summer_data/2022_winter_data/parameter_N6i3j" + str(i) + ".json"
    file = open(fileName,'rb')
    a = pickle.load(file)
    vs.parameters = a
    eng = ex.expect_and_grad_ex(  
        vstate=vs,
        Ô=ha,
        use_covariance=True,
        mutable=False,
        state_list = state_list,
        shift_list = shift_list)
    engs.append(eng[0].mean)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [10]:
engs = np.real(engs)
print(np.argmin(engs))
print(np.min(engs))

10
-1.2216853558727614


In [29]:
state_list = [vs0]
shift_list = [0.3]
c = ex.expect_and_grad_ex(  
        vstate=vs0,
        Ô=ha,
        use_covariance=True,
        mutable=False,
        state_list = state_list,
        shift_list = shift_list)

In [30]:
print(c[0].mean)

(-1.1153078220666248-1.6800175909233708e-05j)


In [11]:
print(engs)

[-1.19703517 -1.22047441 -1.2171219  -1.21839535 -1.22027391 -1.21963791
 -1.21734104 -1.2141546  -1.21197374 -1.21891799 -1.22168536 -1.21830645
 -1.21967047 -1.21828533 -1.22153162 -1.2203774  -1.21952157 -1.21975729
 -1.22021566 -1.22121905 -1.22069189 -1.21517877 -1.21912274 -1.2189631
 -1.21893152 -1.21979111 -1.21987306 -1.22003725 -1.2202064  -1.21982264
 -1.21948052 -1.20132773 -1.21800651 -1.2212997  -1.21984279 -1.2203083
 -1.2196263  -1.21915941 -1.22129264 -1.21914178 -1.22008187 -1.219562
 -1.21985775 -1.21951321 -1.21943651 -1.21928174 -1.22056946 -1.21988856
 -1.21915686 -1.21903302]


In [37]:
print(vs1.expect(ha))

-1.32719+0.00071j ± 0.00030 [σ²=0.00065, R̂=1.0018]
